# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I {**MUKTHAR**, **ABDULWAHEED**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [2]:
sc = SparkContext()
spark =SparkSession(sc)

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [3]:
df_p = pd.read_csv(r'C:\Users\dell\Documents\Explore Data Engineering\3. AWS Cloud Compute\Processing Big Data\stocks\1963\02\04\stocks.csv')

In [4]:
df_p.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,1963-02-04,5.763195,5.763195,5.639040,5.663070,1.360517,27900.0,AA
1,1963-02-04,5.423069,5.423069,5.294228,5.329367,1.256773,29800.0,ARNC
2,1963-02-04,0.656379,0.664609,0.650206,0.650206,0.120512,959600.0,BA
3,1963-02-04,1.583333,1.583333,1.541667,1.572917,0.139002,103200.0,CAT
4,1963-02-04,0.000000,4.101562,4.062500,4.078125,0.066616,116800.0,CVX


In [5]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       20 non-null     object 
 1   Open       20 non-null     float64
 2   High       20 non-null     float64
 3   Low        20 non-null     float64
 4   Close      20 non-null     float64
 5   Adj Close  20 non-null     float64
 6   Volume     20 non-null     float64
 7   stock      20 non-null     object 
dtypes: float64(6), object(2)
memory usage: 1.4+ KB


## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [6]:
from pyspark.sql.types import StructField, StructType, StringType

df_schema = StructType([
    StructField('Date', StringType()),
    StructField('Open', StringType()),
    StructField('High', StringType()),
    StructField('Low', StringType()),
    StructField('Close', StringType()),
    StructField('Adj Close', StringType()),
    StructField('Volume', StringType()),
    StructField('stock', StringType())
])

In [7]:
df_nasdaq = spark.read.csv(r'C:\Users\dell\Documents\Explore Data Engineering\3. AWS Cloud Compute\Processing Big Data\stocks\1962\*\*\*.csv', header=True, schema=df_schema)

In [8]:
df_nasdaq.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- stock: string (nullable = true)



In [9]:
df_nasdaq.show(5, truncate=False)

+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|Date      |Open              |High              |Low               |Close             |Adj Close         |Volume  |stock|
+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|1962-02-19|5.839290142059326 |5.907374858856201 |5.839290142059326 |5.863319873809815 |1.3863292932510376|29900.0 |AA   |
|1962-02-19|5.481634140014648 |5.5284857749938965|5.481634140014648 |5.516772747039795 |1.2804527282714844|32000.0 |ARNC |
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835|0.903292179107666 |0.1614154428243637|619400.0|BA   |
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207|170400.0|CAT  |
|1962-02-19|0.0               |3.5788691043853764|20.0              |3.549107074737549 |0.0565012246370315|273600.0|CVX  |
+----------+----

## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [10]:
# Replacing whitespaces in columns with underscore and put them in lowercase

for column in df_nasdaq.columns:
    df_nasdaq = df_nasdaq.withColumnRenamed(column, '_'.join(column.split()).lower())
    
df_nasdaq.show(5)

+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|      date|              open|              high|               low|             close|         adj_close|  volume|stock|
+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|1.3863292932510376| 29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|1.2804527282714844| 32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|0.1614154428243637|619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207|170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|0.0565012246370315|273600.0|  CVX|
+----------+----

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [11]:
#checking for null values

df_nasdaq.select([F.count(F.when(F.isnull(x), x)).alias(x) for x in df_nasdaq.columns]).show()

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 22|    0|        0|    21|    0|
+----+----+----+---+-----+---------+------+-----+



## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [12]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType, TimestampType, DateType

df_schema = StructType([
    StructField('date', DateType()),
    StructField('open', FloatType()),
    StructField('high', FloatType()),
    StructField('low', FloatType()),
    StructField('close', FloatType()),
    StructField('adj_close', FloatType()),
    StructField('volume', FloatType()),
    StructField('stock', StringType())
])

In [13]:
df_nasdaq = spark.read.csv(r'C:\Users\dell\Documents\Explore Data Engineering\3. AWS Cloud Compute\Processing Big Data\stocks\1962\*\*\*.csv', header=True, schema=df_schema)

In [14]:
df_nasdaq.show(10, truncate=False)

+----------+-----------+-----------+----------+----------+------------+---------+-----+
|date      |open       |high       |low       |close     |adj_close   |volume   |stock|
+----------+-----------+-----------+----------+----------+------------+---------+-----+
|1962-02-19|5.83929    |5.907375   |5.83929   |5.86332   |1.3863293   |29900.0  |AA   |
|1962-02-19|5.481634   |5.528486   |5.481634  |5.5167727 |1.2804527   |32000.0  |ARNC |
|1962-02-19|0.9074074  |0.91563785 |0.89917696|0.9032922 |0.16141544  |619400.0 |BA   |
|1962-02-19|1.6770834  |1.6927084  |1.6614584 |1.6770834 |0.1440587   |170400.0 |CAT  |
|1962-02-19|0.0        |3.578869   |20.0      |3.549107  |0.056501225 |273600.0 |CVX  |
|1962-02-19|0.099767394|0.099767394|0.09820853|0.09820853|0.037543412 |817400.0 |DIS  |
|1962-02-19|0.0        |29.9375    |29.75     |29.9375   |0.49964145  |1600.0   |DTE  |
|1962-02-19|0.0        |9.921875   |9.890625  |9.921875  |0.22499175  |8800.0   |ED   |
|1962-02-19|0.0        |7.083333

## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [15]:
#checking for null values

df_nasdaq.select([F.count(F.when(F.isnull(x), x)).alias(x) for x in df_nasdaq.columns]).show()

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 42|    0|       21|    21|    0|
+----+----+----+---+-----+---------+------+-----+



Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [16]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [17]:
rdd = df_nasdaq.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')

The dataframe is 11.243584 MB


### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [18]:
df_nasdaq.coalesce(partitions).write.format("parquet").mode("append").save("parquet/1962/data_ingestion.parquet")

In [19]:
spark.stop()